CNN modelleri 4 boyutlu bir veri yapısı bekler: (Örnek Sayısı, Yükseklik, Genişlik, Kanal Sayısı).
- 28, 28: Resmin boyutları.
- 1: Kanal sayısı (Grayscale yani siyah-beyaz olduğu için 1. Eğer renkli/RGB olsaydı 3 olacaktı).

In [4]:
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [5]:
#Reshape: Görüntüyü 28,28,1 boyutuna getiriyorum
X_train_cnn = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test_cnn = X_test.reshape(X_test.shape[0], 28, 28, 1)

#Normalize: Görüntüyü 0-1 aralığına getiriyorum
X_train_cnn = X_train_cnn.astype('float32') / 255.0
X_test_cnn = X_test_cnn.astype('float32') / 255.0


Conv2D (Convolutional Layer): Görüntünün üzerinde küçük bir pencere (filtre) gezdirerek kenarları, köşeleri ve desenleri yakalar.

MaxPooling2D: Görüntünün boyutunu yarıya indirir. En önemli özellikleri (en yüksek piksel değerlerini) seçer ve hesaplama yükünü azaltır.

Flatten: Artık özellikler yakalandığına göre, 2D matrisi tekrar MLP'nin anlayacağı düz bir liste (vektör) haline getirir.

Dense: Klasik yapay sinir ağı katmanıdır; öğrenilen özellikleri kullanarak sınıflandırma yapar.



In [6]:
from tensorflow.keras import layers,models

model = models.Sequential([
    layers.Conv2D(21, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(42, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

#neden iki kere conv ve maxpooling? : Yapay sinir ağları bir resmi tek seferde bütünüyle anlayamaz.
#İlk katmanlar basit özellikleri (kenarlar, köşeler) tanır, sonraki katmanlar bu özellikleri birleştirerek daha karmaşık desenleri tanır.
#Her MaxPooling işlemi resmin boyutunu yarıya indirir.
#filtre sayısı neden artıyor? : İlk katmanda 21 filtre, ikinci katmanda 42 filtre kullanarak modelin daha fazla özellik öğrenmesini sağlıyoruz.
#Daha fazla filtre, modelin daha fazla "detaylı desen" tutabilmesini sağlar.

/Users/beyzahiz/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(
    optimizer='adam', #Modelin hata oranını (loss) düşürmek için ağırlıkları (weights) nasıl güncelleyeceğini belirleyen algoritmadır.
    loss='sparse_categorical_crossentropy',
    #Neden "Sparse" olanı seçtik?: Senin etiketlerin (y_train) [5, 0, 4...] gibi tamsayılardan oluşuyor. 
    #Eğer bu rakamlar tamsayı formatındaysa sparse_categorical_crossentropy en uygunudur.
    metrics=['accuracy']
)